# AI Career Advisor Agent - Interactive Intake

This notebook implements an interactive career advisor using:
- **Single Intake Agent** for conversational information gathering
- **SequentialAgent** for the main pipeline (research + mentoring)
- **Memory** for cross-session context

## Architecture
```
Stage 1: Intake (Multi-turn conversation)
User → Intake Agent (asks questions one at a time)
      ↓ (repeat until profile complete)
      Saves profile to memory

Stage 2: Main Pipeline (Single turn)
User triggers → Research Agent → Mentor Agent
```

**Key Insight:** We use TWO separate runners because intake needs multiple turns with the user, while research+mentor run in one go.

In [ ]:
# Import dependencies
from dotenv import load_dotenv
import os

load_dotenv()

from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.runners import Runner
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search, preload_memory, FunctionTool
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService

print("✅ ADK components imported successfully.")

In [2]:
# Setup API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
else:
    print("🔑 Authentication Error: 'GOOGLE_API_KEY' not found in .env file.")

✅ Gemini API key setup complete.


In [3]:
# Configuration
APP_NAME = "career_advisor"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

# Retry configuration
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ Configuration loaded.")

✅ Configuration loaded.


In [4]:
# Helper function for running sessions
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
    show_all_agents: bool = False,
):
    """Run queries in a session and display agent responses."""
    print(f"\n{'='*60}")
    print(f"Session: {session_name}")
    print(f"{'='*60}")

    app_name = runner_instance.app_name

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
        print("✅ New session created")
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
        print("✅ Existing session retrieved")

    if user_queries:
        if isinstance(user_queries, str):
            user_queries = [user_queries]

        for query in user_queries:
            print(f"\n👤 User: {query}")
            print(f"{'-'*60}")

            query_content = types.Content(role="user", parts=[types.Part(text=query)])

            responses = []

            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query_content
            ):
                if event.content and event.content.parts:
                    text = event.content.parts[0].text
                    if text and text != "None":
                        agent_name = getattr(event, 'author', 'Agent')

                        if show_all_agents:
                            print(f"\n🤖 {agent_name}:")
                            print(text)
                        else:
                            responses.append({'agent': agent_name, 'text': text})

            # If not showing all, print only final response
            if not show_all_agents and responses:
                last = responses[-1]
                print(f"\n🤖 {last['agent']}:")
                print(last['text'])
    else:
        print("⚠️  No queries provided!")

print("✅ Helper function defined.")

✅ Helper function defined.


In [5]:
# Memory callback
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    try:
        agent_name = callback_context._invocation_context.agent.name
        session = callback_context._invocation_context.session

        await callback_context._invocation_context.memory_service.add_session_to_memory(session)

        print(f"💾 Saved {agent_name} output to memory")
    except Exception as e:
        print(f"⚠️  Warning: Failed to save to memory: {e}")

print("✅ Callback created.")

✅ Callback created.


## Tool Functions

In [ ]:
# Validation function
def validate_user_profile(
    current_role: str,
    years_experience: int,
    target_role: str,
    time_commitment_hours: int = 10
) -> dict:
    """
    Validates that all required user profile information has been collected.
    
    Args:
        current_role: The user's current job role
        years_experience: Years of experience in current role (0-50)
        target_role: The role the user wants to transition to
        time_commitment_hours: Hours per week available for learning (1-168)
    
    Returns:
        Dictionary with status and message
    """
    # Validate inputs
    if not current_role or len(current_role.strip()) < 2:
        return {"status": "incomplete", "message": "Current role is missing or invalid"}
    
    if not target_role or len(target_role.strip()) < 2:
        return {"status": "incomplete", "message": "Target role is missing or invalid"}
    
    if not isinstance(years_experience, int) or years_experience < 0 or years_experience > 50:
        return {"status": "invalid", "message": "Years of experience must be a number between 0 and 50"}
    
    if not isinstance(time_commitment_hours, int) or time_commitment_hours < 1 or time_commitment_hours > 168:
        return {"status": "invalid", "message": "Time commitment must be between 1 and 168 hours per week"}
    
    # All validations passed
    return {
        "status": "complete",
        "message": f"✅ Profile complete! Ready to research transition from {current_role} to {target_role}.",
        "profile": {
            "current_role": current_role,
            "years_experience": years_experience,
            "target_role": target_role,
            "time_commitment_hours": time_commitment_hours
        }
    }

print("✅ Validation function created.")

## Agent Definitions

In [ ]:
# Intake Agent - Conversational information gathering
intake_agent = LlmAgent(
    name="intake_agent",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    instruction="""
    You are an Intake Agent for career transition planning.
    
    Required information (4 items):
    1. current_role - User's current job/position
    2. years_experience - Years in current role (must be a NUMBER)
    3. target_role - Role they want to transition to
    4. time_commitment_hours - Hours per week for learning (must be a NUMBER, default: 10)
    
    PROCESS:
    - Review the conversation history to see what information has already been provided
    - Check preload_memory for any past information
    - Identify what's still missing
    
    IF missing information:
        - Ask ONE friendly question for the next missing piece
        - Be conversational, warm, and encouraging
        - Don't call any tools yet
    
    IF you have all 4 pieces:
        - Call validate_user_profile() with all the gathered data
        - If validation returns "complete":
           * Output a friendly summary of the complete profile
           * Inform the user you're ready to help them with their transition
        - If validation returns "incomplete"/"invalid":
           * Ask for the missing or incorrect information
    
    CRITICAL RULES:
    - years_experience and time_commitment_hours MUST be integers (whole numbers)
    - Convert text like "five years" to 5, "ten hours" to 10
    - Only call validate_user_profile() when you have ALL 4 pieces
    - Ask questions ONE AT A TIME
    - Be friendly and natural, not robotic
    """,
    tools=[preload_memory, FunctionTool(validate_user_profile)],
    output_key="user_profile",
    after_agent_callback=auto_save_to_memory,
)

print("✅ Intake agent created.")

In [9]:
# Intake Loop - Combines question and validator agents
intake_loop = LoopAgent(
    name="IntakeLoop",
    sub_agents=[question_agent, validator_agent],
    max_iterations=10,  # Safety limit to prevent infinite loops
)

print("✅ Intake loop created.")

✅ Intake loop created.


In [11]:
# Mentor Agent
mentor_agent = LlmAgent(
    name="mentor_agent",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    instruction="""
    You are a Career Mentor Agent.
    
    USER PROFILE:
    {user_profile}
    
    RESEARCH FINDINGS:
    {research_summary}
    
    Create a personalized, actionable transition plan tailored to the user's profile.
    
    Structure your plan:
    
    ## Phase 1: Foundation (Months 1-3)
    - Specific skills to learn first (prioritized for their background)
    - Recommended courses/resources with links
    - Time commitment strategy (based on their available hours/week)
    - Quick wins to build confidence
    
    ## Phase 2: Building Portfolio (Months 4-6)
    - Concrete projects to build (relevant to their experience)
    - GitHub repositories to create
    - Communities to join (specific names)
    - Networking strategies
    
    ## Phase 3: Job Search (Months 6-9)
    - Resume updates needed
    - Where to apply (companies, job boards)
    - Interview preparation tips
    - Portfolio presentation strategies
    
    ## Milestones & Checkpoints
    - Monthly goals with measurable outcomes
    - How to measure progress
    - Red flags and when to adjust
    
    ## Leveraging Your Background
    - How to translate their existing skills
    - Unique advantages from their current position
    - How their experience is an asset
    
    Be specific, realistic, and encouraging. Use actual course names and platforms.
    Tone: supportive but practical - acknowledge challenges while emphasizing achievability.
    """,
    output_key="career_advice",
)

print("✅ Mentor agent created.")

✅ Mentor agent created.


## Pipeline Assembly

In [12]:
# Root Agent - Sequential pipeline with intake loop first
root_agent = SequentialAgent(
    name="CareerAdvisorPipeline",
    sub_agents=[intake_loop, research_agent, mentor_agent],
)

print("✅ Root agent created with 3-stage pipeline:")
print("   1. Intake Loop (question → validate → repeat until complete)")
print("   2. Research Agent (gather career transition info)")
print("   3. Mentor Agent (create personalized action plan)")

✅ Root agent created with 3-stage pipeline:
   1. Intake Loop (question → validate → repeat until complete)
   2. Research Agent (gather career transition info)
   3. Mentor Agent (create personalized action plan)


In [13]:
# Initialize services and runner
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Runner initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Single runner for entire pipeline")

✅ Runner initialized!
   - Application: career_advisor
   - User: default
   - Single runner for entire pipeline


# Main Pipeline - Sequential agent for research + mentor
pipeline_agent = SequentialAgent(
    name="CareerAdvisorPipeline",
    sub_agents=[research_agent, mentor_agent],
)

print("✅ Pipeline agent created with 2-stage pipeline:")
print("   1. Research Agent (gather career transition info)")
print("   2. Mentor Agent (create personalized action plan)")

In [ ]:
# Initialize services
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

# Create TWO runners
# Runner 1: For intake (multi-turn conversation to gather profile)
intake_runner = Runner(
    agent=intake_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

# Runner 2: For main pipeline (single-turn research + mentoring)
pipeline_runner = Runner(
    agent=pipeline_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Two runners initialized!")
print(f"   - intake_runner: Multi-turn conversation to gather user profile")
print(f"   - pipeline_runner: Single-turn research + mentoring")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")

In [15]:
# Test 2: Continue answering questions in the SAME session
# Answer whatever the agent asked in the previous cell
await run_session(
    runner,
    "Product Manager",  # Example answer to "What's your current role?"
    session_name="interactive-test-1",  # Same session!
    show_all_agents=True,
)


Session: interactive-test-1
✅ Existing session retrieved

👤 User: Product Manager
------------------------------------------------------------

🤖 question_agent:
That's fantastic! Side projects are a powerful way to learn and demonstrate your product thinking. For someone with a software engineering background, leveraging your technical skills to build something innovative is a great approach.

Here are a few product ideas, keeping in mind your background and the goal of showcasing product management skills:

1.  **Developer Productivity Tools:** As a software engineer, you understand the pain points of the development process. Consider building a tool that:
    *   Automates a tedious task (e.g., code generation, test case creation, environment setup).
    *   Improves code review efficiency.
    *   Helps with debugging or monitoring.
    *   Facilitates collaboration within engineering teams.
    *   **Product Thinking Angle:** You'd be identifying a "customer" (other developers), 


🤖 validator_agent:
I'll focus on Python first.

🤖 question_agent:
Excellent! Python is a great choice, with a rich ecosystem for this kind of tool. Focusing on one language for the MVP makes perfect sense.

So, you're building an automated API documentation generator for Python, targeting individual developers and small teams, with an MVP that generates docs from code comments and function signatures.

Do you want to explore resources for learning more about product management specifically related to building developer tools, or would you prefer to dive into tools and techniques for building your MVP?

🤖 validator_agent:
I'd like to learn more about product management for developer tools.

🤖 question_agent:
Great choice! Product management for developer tools (often called "DevTools PM") has its own unique nuances. It's about understanding the developer as a customer.

Here are some key considerations for PMing DevTools:

*   **Developer Empathy is Paramount:** You *are* the user, or 

CancelledError: 

In [ ]:
# STAGE 1: Start intake conversation
# The intake agent will ask you questions one at a time
await run_session(
    intake_runner,  # Use intake_runner for gathering info
    "I want to change careers",
    session_name="interactive-test-1",
    show_all_agents=False,
)

In [ ]:
# STAGE 1: Continue answering intake questions
# Answer whatever the agent asked in the previous cell
await run_session(
    intake_runner,  # Same runner, same session
    "Product Manager",  # Example answer to "What's your current role?"
    session_name="interactive-test-1",
    show_all_agents=False,
)

# STAGE 1: Continue with more answers
await run_session(
    intake_runner,
    "5 years",  # Years of experience
    session_name="interactive-test-1",
    show_all_agents=False,
)

In [ ]:
# STAGE 1: Provide remaining info (can provide multiple pieces at once)
await run_session(
    intake_runner,
    "Data Scientist and I can dedicate 10 hours per week",
    session_name="interactive-test-1",
    show_all_agents=False,
)
# Once the agent validates the profile, it will confirm completion

In [ ]:
# STAGE 2: Run the main pipeline (after intake is complete)
# This uses the profile saved in memory to run research + mentoring
await run_session(
    pipeline_runner,  # Switch to pipeline_runner
    "Now create my personalized career transition plan",
    session_name="interactive-test-1",  # Same session to access the profile
    show_all_agents=False,
)

# Alternative: Provide everything upfront (non-interactive)
# If you give all context in one message, you can go directly to pipeline_runner
await run_session(
    pipeline_runner,  # Go straight to pipeline runner
    "I am a Product Manager with 5 years of experience. I want to transition to Data Science and can dedicate 10 hours per week to learning.",
    session_name="all-in-one-test",
    show_all_agents=False,
)

## How This Works

### **Two-Runner Approach:**

**Why two runners?**
- **Intake** needs multiple back-and-forth turns with the user (conversational)
- **Research + Mentor** runs in one shot once they have the profile

**Stage 1: Intake (Multi-turn)**
```python
await run_session(intake_runner, "I want to change careers", "session-1")
# Agent asks: "What's your current role?"

await run_session(intake_runner, "Product Manager", "session-1")
# Agent asks: "How many years of experience?"

# ... continues until profile is complete and validated
```

**Stage 2: Main Pipeline (Single turn)**
```python
await run_session(pipeline_runner, "Create my plan", "session-1")
# Research + Mentor agents run automatically with the saved profile
```

### **Benefits:**

✅ **Natural Conversation** - Intake agent asks one question at a time

✅ **Validated** - Can't proceed without complete, valid profile data

✅ **Flexible** - Can also skip intake if user provides everything upfront

✅ **Memory-Aware** - Uses preload_memory to check past conversations

✅ **Clear Separation** - Intake for gathering, pipeline for execution